# Creating your 1st graph and running it in a session

In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [2]:
session = tf.Session()
session.run(x.initializer)
session.run(y.initializer)
result = session.run(f)
print(result)
session.close()

42


In [3]:
# To avoid repeating session.run()
with tf.Session() as session:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

In [4]:
# Avoid repeating initializer, use a global variable.
# It also initializes the variables.
init = tf.global_variables_initializer()
with tf.Session() as session:
    init.run()
    result = f.eval()
result

42

In [5]:
# Interactive session sets the actual session as a default session
session = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
session.close()

42


# Managing graphs

In [6]:
x1 = tf.Variable(3)
with tf.Session() as session:
    print(x1.graph is tf.get_default_graph())

True


In [7]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(1)
    print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


# Lifecycle of a Node Value

In [8]:
# tf makes two calls to w & x
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as session:
    print(y.eval())
    print(z.eval())

10
15


In [9]:
# Evaluate both expression to execute w & x once
with tf.Session() as session:
    y_val, z_val = session.run([y, z])
    print(y_val)
    print(z_val)

10
15


# Linear Regression with TensorFlow

In [10]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as session:
    thetha_value = theta.eval()
print(thetha_value.shape)

(9, 1)


# Implementing Gradient Descent

In [16]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [18]:
reset_graph()
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print ("Epoch", epoch, "MSE = ", mse.eval())
        session.run(training_op)
    best_theta = theta.eval()

ValueError: Dimensions must be equal, but are 8 and 9 for 'predictions' (op: 'MatMul') with input shapes: [20640,8], [9,1].